# CentraleSupelec - Natural language processing
# Practical session n°7

## Natural Language Inferencing (NLI): 

(NLI) is a classical NLP (Natural Language Processing) problem that involves taking two sentences (the premise and the hypothesis ), and deciding how they are related- if the premise *entails* the hypothesis, *contradicts* it, or *neither*.

Ex: 


| Premise | Label | Hypothesis |
| --- | --- | --- |
| A man inspects the uniform of a figure in some East Asian country. | contradiction | The man is sleeping. |
| An older and younger man smiling. | neutral | Two men are smiling and laughing at the cats playing on the floor. |
| A soccer game with multiple males playing. | entailment | Some men are playing a sport. |

### Stanford NLI (SNLI) corpus

In this labwork, I propose to use the Stanford NLI (SNLI) corpus ( https://nlp.stanford.edu/projects/snli/ ), available in the *Datasets* library by Huggingface.

    from datasets import load_dataset
    snli = load_dataset("snli")
    #Removing sentence pairs with no label (-1)
    snli = snli.filter(lambda example: example['label'] != -1) 

## Subject

You are asked to provide an operational Jupyter notebook that performs the task of NLI. For that, you need to tackle the following aspects of the problem:

1. Loading and preprocessing the data
2. Designing a PyTorch model that, given two sentences, decides how they are related (*entails*, *contradicts* or *neither*.)
3. Training and evaluating the model using appropriate metrics
4. (Optional) Allowing to play with the model (forward user sentences and visualize the prediction easily)
5. (Optional) Providing visual insight about the model (i.e. visualizing the attention if your model is using attention)

Although it is not mandatory, I suggest that you use a transformer model to perform the task. For that, you can use the *Transformer* library by Huggingface.

## Evaluation

The evaluation will be based on several criteria:

- Clarity and readability of the notebook. The notebook is the report of you project. Make it easy and pleasant to read.
- Justification of implementation choices (i.e. the network, the cost funtion, the optimizer, ...)
- Quality of the code. The various deeplearning and NLP labworks provide many example of good practices for designing experiments with neural networks. Use them as inspirational examples!

## Additional recommendations

- You are not seeking to publish a research paper! I'm not expecting state-of-the-art results! The idea of this labwork is to assess that you have integrated the skills necessary to handle textual data using deep neural network techniques.

- This labwork will be evaluated but we are still here to help you! Don't hesitate to request our help if you are stuck.

- If you intend to use BERT based models, let me give you an advice. The bert-base-* models available in *Transformers* need more than 12Go to be fine-tuned on GPU. To avoid memory issues, you can use several solutions: 

    - Use a lighter BERT based model such as DistilBERT, ALBERT, ...
    - Train a classification model on top of BERT, whithout fine-tuning it (i.e. freezing BERT weights)

## Huggingface documentations

In case you want to use the huggingface *Datasets* and *Transformer* libraries (which I advice), here are some useful documentation pages:

- Dataset quick tour

    https://huggingface.co/docs/datasets/quicktour.html
    
- Documentation on data preprocessing for transformers

    https://huggingface.co/transformers/preprocessing.html
    
- Transformer Quick tour (with distilbert example for classification).

    https://huggingface.co/transformers/quicktour.html
    


## Part 0 : Imports

In [1]:
from nltk.tokenize import word_tokenize 
import pandas as pd
import os
from datasets import load_dataset
from transformers import AutoTokenizer, BertForSequenceClassification,AutoModelForSequenceClassification, DistilBertConfig, DistilBertTokenizer, DistilBertForSequenceClassification, BertTokenizer, BertModel
import time
import multiprocessing
from tqdm import tqdm
import torch
from torch import nn

## Part 0 bis : Variables

In [2]:
BATCH_SIZE = 64
device = 'cuda' if torch.cuda.is_available() else 'cpu'
learning_rate = 1e-5
epocs = 5
cpu_nb=multiprocessing.cpu_count()

## Part 1 : Load data / tokenizer and preprocess


Dans cette première partie, on va télécharger le dataset et le stocker dans un dataframe pandas. Ensuite, on effectura un preprocessing des données en effectuant une tokenization des phrases des corpus.

In [3]:
snli = load_dataset("snli")
#Removing sentence pairs with no label (-1)
snli = snli.filter(lambda example: example['label'] != -1)

Reusing dataset snli (/usr/users/gpusdi1/gpusdi1_34/.cache/huggingface/datasets/snli/plain_text/1.0.0/bb1102591c6230bd78813e229d5dd4c7fbf4fc478cec28f298761eb69e5b537c)
Loading cached processed dataset at /usr/users/gpusdi1/gpusdi1_34/.cache/huggingface/datasets/snli/plain_text/1.0.0/bb1102591c6230bd78813e229d5dd4c7fbf4fc478cec28f298761eb69e5b537c/cache-9fd499264a29715c.arrow
Loading cached processed dataset at /usr/users/gpusdi1/gpusdi1_34/.cache/huggingface/datasets/snli/plain_text/1.0.0/bb1102591c6230bd78813e229d5dd4c7fbf4fc478cec28f298761eb69e5b537c/cache-d42e8f7826ad1311.arrow
Loading cached processed dataset at /usr/users/gpusdi1/gpusdi1_34/.cache/huggingface/datasets/snli/plain_text/1.0.0/bb1102591c6230bd78813e229d5dd4c7fbf4fc478cec28f298761eb69e5b537c/cache-c2a8e9801b4a0855.arrow


In [4]:
snli

DatasetDict({
    test: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 9824
    })
    train: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 549367
    })
    validation: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 9842
    })
})

In [5]:
snli['train']['premise'][1]

'A person on a horse jumps over a broken down airplane.'

In [6]:
snli['train']['hypothesis'][1]

'A person is at a diner, ordering an omelette.'

In [7]:
snli['train']['label'][1]

2

In [8]:
train = snli['train']
validation = snli['validation']
test = snli['test']

In [9]:
print(train.shape)
print(validation.shape)
print(test.shape)

(549367, 3)
(9842, 3)
(9824, 3)


In [10]:

disttokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')
distmodel = DistilBertForSequenceClassification.from_pretrained("distilbert-base-cased")


Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier

In [11]:
disttokenizer

PreTrainedTokenizer(name_or_path='distilbert-base-cased', vocab_size=28996, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [12]:
def dist_encode(dataset):
    return(disttokenizer(dataset['premise'],dataset['hypothesis'],padding='max_length',truncation=True))

In [13]:
train = train.map(dist_encode, batched=True,batch_size = len(train))
test = test.map(dist_encode, batched=True,batch_size = len(test))
validation = validation.map(dist_encode, batch_size = len(validation),batched=True)

Loading cached processed dataset at /usr/users/gpusdi1/gpusdi1_34/.cache/huggingface/datasets/snli/plain_text/1.0.0/bb1102591c6230bd78813e229d5dd4c7fbf4fc478cec28f298761eb69e5b537c/cache-83b2de01fcc91fce.arrow
Loading cached processed dataset at /usr/users/gpusdi1/gpusdi1_34/.cache/huggingface/datasets/snli/plain_text/1.0.0/bb1102591c6230bd78813e229d5dd4c7fbf4fc478cec28f298761eb69e5b537c/cache-86941808979b15b4.arrow
Loading cached processed dataset at /usr/users/gpusdi1/gpusdi1_34/.cache/huggingface/datasets/snli/plain_text/1.0.0/bb1102591c6230bd78813e229d5dd4c7fbf4fc478cec28f298761eb69e5b537c/cache-4816f8cb9004eac1.arrow


In [14]:
print(train)

Dataset({
    features: ['attention_mask', 'hypothesis', 'input_ids', 'label', 'premise'],
    num_rows: 549367
})


In [15]:
train = train.map(lambda examples: {'labels': examples['label']}, batched=True, num_proc = 7)
validation = validation.map(lambda examples: {'labels': examples['label']}, batched=True, num_proc = 7)
test = test.map(lambda examples: {'labels': examples['label']}, batched=True, num_proc = 7)

Loading cached processed dataset at /usr/users/gpusdi1/gpusdi1_34/.cache/huggingface/datasets/snli/plain_text/1.0.0/bb1102591c6230bd78813e229d5dd4c7fbf4fc478cec28f298761eb69e5b537c/cache-b0adb1d9f1fdfa9c.arrow
Loading cached processed dataset at /usr/users/gpusdi1/gpusdi1_34/.cache/huggingface/datasets/snli/plain_text/1.0.0/bb1102591c6230bd78813e229d5dd4c7fbf4fc478cec28f298761eb69e5b537c/cache-e92a8a59bd2ba4cf.arrow
Loading cached processed dataset at /usr/users/gpusdi1/gpusdi1_34/.cache/huggingface/datasets/snli/plain_text/1.0.0/bb1102591c6230bd78813e229d5dd4c7fbf4fc478cec28f298761eb69e5b537c/cache-16348ae440e1c068.arrow
Loading cached processed dataset at /usr/users/gpusdi1/gpusdi1_34/.cache/huggingface/datasets/snli/plain_text/1.0.0/bb1102591c6230bd78813e229d5dd4c7fbf4fc478cec28f298761eb69e5b537c/cache-f3d0ad0d2f1d252a.arrow
Loading cached processed dataset at /usr/users/gpusdi1/gpusdi1_34/.cache/huggingface/datasets/snli/plain_text/1.0.0/bb1102591c6230bd78813e229d5dd4c7fbf4fc478cec

In [16]:
train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
validation.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

train_dataloader = torch.utils.data.DataLoader(train, batch_size=BATCH_SIZE, num_workers=multiprocessing.cpu_count())
val_dataloader = torch.utils.data.DataLoader(validation, batch_size=BATCH_SIZE,num_workers=multiprocessing.cpu_count())
test_dataloader = torch.utils.data.DataLoader(train, batch_size=BATCH_SIZE,num_workers=multiprocessing.cpu_count())

In [17]:
next(iter(train_dataloader))

{'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'input_ids': tensor([[ 101,  138, 1825,  ...,    0,    0,    0],
         [ 101,  138, 1825,  ...,    0,    0,    0],
         [ 101,  138, 1825,  ...,    0,    0,    0],
         ...,
         [ 101, 5651, 1107,  ...,    0,    0,    0],
         [ 101, 5651, 1107,  ...,    0,    0,    0],
         [ 101, 5651, 1107,  ...,    0,    0,    0]]),
 'labels': tensor([1, 2, 0, 1, 0, 2, 2, 0, 1, 1, 2, 1, 1, 2, 0, 1, 2, 0, 0, 2, 1, 1, 2, 0,
         2, 0, 1, 1, 2, 0, 1, 0, 2, 2, 1, 0, 2, 0, 1, 1, 0, 2, 1, 0, 0, 0, 1, 2,
         2, 0, 1, 2, 0, 1, 2, 1, 0, 1, 2, 0, 0, 2, 1, 0])}

In [18]:
print(train['premise'][1])
print(disttokenizer.decode(train['input_ids'][1]))

A person on a horse jumps over a broken down airplane.
[CLS] A person on a horse jumps over a broken down airplane. [SEP] A person is at a diner, ordering an omelette. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD

Entrainons maintenant le modèle distilbert

## Part 4 : Train

In [24]:
def train(model,clip):
    totloss=0
    model.train().to(device)
    optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
    for i, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = nn.CrossEntropyLoss().to(device)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        totloss+= loss.item()
        optimizer.step()
        optimizer.zero_grad()
        if i % 100 == 0:
            print(f"train_loss: {loss}")
    lossavg = totloss/len(train_dataloader)
    print(f"Loss: {lossavg}")
    return lossavg

def val(model):
    model.train().to(device)    
    model.eval()
    totloss = 0
    optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
    for i, batch in enumerate(tqdm(val_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs[0]
        total_loss+=loss.item()
        if i % 100 == 0:
            print(f"Val loss: {loss:.3f}")
    lossavg = totloss/len(train_dataloader)
    print(f"Loss: {lossavg}")
    return lossavg



In [26]:
#https://github.com/12860/NLP/blob/master/Seq2seq.py

best_validation_loss = float('inf')
clip=1
for epoch in range(epocs):
    start_time = time.time()    
    train_loss = train(distmodel,clip)
    valid_loss= val(distmodel)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'dist_model.pt')
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f}')
    

  0%|          | 0/8584 [00:00<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 96.00 MiB (GPU 0; 10.76 GiB total capacity; 9.69 GiB already allocated; 13.06 MiB free; 9.73 GiB reserved in total by PyTorch)


## Part 5 : Evaluer => afficher precision, recall, F1
tester avec des phrases

In [ ]:
model_best = load_state_dict(torch.load('dist_model.pt'))